**Connect google drive**

In [0]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**1) Clone the Darknet**



In [0]:
!git clone https://github.com/AlexeyAB/darknet

**2) Compile Darknet using Nvidia GPU**


In [0]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V4**

In [0]:
!cp cfg/yolov4-tiny.cfg cfg/yolov4_training.cfg

In [0]:
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov4_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 174000/' cfg/yolov4_training.cfg
!sed -i 's/steps=400000,450000/steps=139200,156600/' cfg/yolov4_training.cfg
!sed -i '220 s@classes=80@classes=87@' cfg/yolov4_training.cfg
!sed -i '269 s@classes=80@classes=87@' cfg/yolov4_training.cfg
!sed -i '212 s@filters=255@filters=276@' cfg/yolov4_training.cfg
!sed -i '263 s@filters=255@filters=276@' cfg/yolov4_training.cfg

In [0]:
!cp /mydrive/yolov4/train.txt data/
!cp /mydrive/yolov4/obj.names data/
!echo -e 'classes= 87\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov4' > data/obj.data
!mkdir data/obj

**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov4" on Google Drive

In [0]:
!unzip /mydrive/yolov4/images.zip -d data/obj

**6) Start the training**

In [0]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov4_training.cfg /mydrive/yolov4/yolov4_training_last.weights -dont_show